In [1]:
import numpy as np
import pandas as pd 
import bokeh.io
import bokeh.plotting
import scikit_posthocs as posthoc

bokeh.io.output_notebook()

Loading BokehJS ...

# Extended Data Figure 2: Administering AAV via intra-cisterna magna administration.

In [2]:
df_lp = pd.read_csv("https://github.com/GradinaruLab/CAP-Mac/raw/main/raw-data/extended-data-fig2.csv")

############### CALCULATE FOLD-CHANGE  ###################
# Initialize empty DataFrame that we will populate
df_delta = pd.DataFrame()

for _, group in df_lp.groupby(["age", "animal id", "tissue", "tissue id", "xna"]):
    aav9_enrichment=group.loc[group["variant"]=="AAV9"]["count"]
    group["fold change"] = group["count"].transform(lambda x: x / aav9_enrichment)
    
    df_delta = pd.concat([df_delta, group])
############### CALCULATE FOLD-CHANGE  ###################

######## Make some changes for plotting neatness ############
# Specify columns as categorical and set order
df_delta["variant"] = pd.Categorical(df_delta["variant"], ["AAV9", "AAV-PHP.eB", "AAV.CAP-B2", "AAV.CAP-B10", 
                                                           "AAV.CAP-B22", "AAV9.452sub-LUNG1", "AAV.CAP-Mac", "AAV.CAP-C2"])
df_delta["animal id"] = pd.Categorical(df_delta["animal id"], ["RM-003", "RM-004"])
df_delta["xna"] = pd.Categorical(df_delta["xna"], ["DNA", "RNA"])

# Sort and reset index
df_delta = df_delta.sort_values(["variant", "animal id"]).reset_index(drop=True)
######## Make some changes for plotting neatness ############

############### CALCULATE SUMMARY STATS ###################
cols = ["variant", "animal id", "tissue", "tissue id", "xna", "fold change"]
df_lp_infant = df_delta[((df_delta["age"]=="infant") & 
                         (df_delta["animal id"]!="virus"))].reset_index(drop=True)

df_lp_infant["cats"] = df_lp_infant.apply(lambda x: (x["xna"], x["variant"]), axis=1)

df_lp_mean = df_lp_infant.groupby(["animal id", "tissue", "variant", "xna"]).agg(animal_mean_fold_change=("fold change", "mean"), 
                                                                                 animal_std_fold_change=("fold change", "std"),
                                                                                 n=("fold change", "count"))
df_lp_mean = df_lp_mean.fillna(0).reset_index()
df_lp_mean["upper"] = df_lp_mean["animal_mean_fold_change"] + df_lp_mean["animal_std_fold_change"] / np.sqrt(df_lp_mean["n"])
df_lp_mean["lower"] = df_lp_mean["animal_mean_fold_change"] - df_lp_mean["animal_std_fold_change"] / np.sqrt(df_lp_mean["n"])

df_lp_infant = pd.merge(df_lp_infant, df_lp_mean, on=["animal id", "tissue", "variant", "xna"])

df_lp_mean = df_lp_mean.groupby(["tissue", "variant", "xna"]).agg(tissue_mean_fold_change=("animal_mean_fold_change", "mean"))
df_lp_mean = df_lp_mean.fillna(0).reset_index()
############### CALCULATE SUMMARY STATS ###################

df_lp_infant = pd.merge(df_lp_infant, df_lp_mean, on=["tissue", "variant", "xna"])

######## Make some changes for plotting neatness ############
x1=np.arange(0,100,4)[:8]
x2=np.arange(1,100,4)[:8]

a=df_lp_infant[["xna", "variant", "animal id"]].drop_duplicates().sort_values(by=["xna", "variant", "animal id"]).reset_index(drop=True)
rna = a.loc[a["xna"]=="RNA"].reset_index(drop=True)
dna = a.loc[a["xna"]=="DNA"].reset_index(drop=True)
dna["base"] = np.sort(np.concatenate([x1, x2]))
rna["base"] = np.sort(np.concatenate([x1, x2]))+9+np.max(np.sort(np.concatenate([x1, x2])))
a=pd.concat([dna, rna])

df_lp_infant = pd.merge(df_lp_infant, a, on=["xna", "variant", "animal id"])
df_lp_infant = df_lp_infant.sort_values(by=["xna", "variant"])
######## Make some changes for plotting neatness ############

df_lp_infant.head()

,variant,animal id,age,tissue id,xna,route,tissue,count,fold change,cats,animal_mean_fold_change,animal_std_fold_change,n,upper,lower,tissue_mean_fold_change,base
0,AAV9,RM-003,infant,1,DNA,lp,brain,12,1.0,"(DNA, AAV9)",1.0,0.0,10,1.0,1.0,1.0,0
1,AAV9,RM-003,infant,2,DNA,lp,brain,144,1.0,"(DNA, AAV9)",1.0,0.0,10,1.0,1.0,1.0,0
2,AAV9,RM-003,infant,3,DNA,lp,brain,431,1.0,"(DNA, AAV9)",1.0,0.0,10,1.0,1.0,1.0,0
3,AAV9,RM-003,infant,4,DNA,lp,brain,159,1.0,"(DNA, AAV9)",1.0,0.0,10,1.0,1.0,1.0,0
4,AAV9,RM-003,infant,5,DNA,lp,brain,22079,1.0,"(DNA, AAV9)",1.0,0.0,10,1.0,1.0,1.0,0


## Extended Data Figure 2a - Brain AAV barcode quantification (intra-cisterna magna delivery) 

In [3]:
############### BOKEH FIGURE SETTINGS ###################
tissue = "brain"
df_plot = df_lp_infant[df_lp_infant["tissue"]==tissue][["variant", "animal id", "xna", "tissue", "cats", 
                                                        "animal_mean_fold_change", "tissue_mean_fold_change",
                                                        "upper", "lower", "base"]].drop_duplicates().reset_index(drop=True)
x_range_brain = list(df_plot["cats"].unique())
animal_id = ["RM-003", "RM-004"]
marker_colors = list(bokeh.palettes.Colorblind[5])
markers = ["circle_cross", "diamond_dot", "square_cross", "plus", "star"] 

figure_width = 700
figure_height = 350
bar_width = .75
bar_fill_color="gray"
jitter_width = 0.3
marker_size = 10
bar_line_width = 1

error_size=5
error_line_width=.25
marker_line_width=1

p = bokeh.plotting.figure(title="Brain AAV barcode quantification (infant rhesus macaque; intra-cisterna magna delivery)", height=figure_height, width=figure_width)

p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=np.pi/4
p.axis.major_tick_in = 0
p.axis.major_label_text_color="#000000"
p.axis.major_label_text_font_size="12pt"
p.axis.major_label_standoff=5
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [4]:
p.scatter(x=bokeh.transform.dodge("base", -jitter_width, range=p.x_range), y="animal_mean_fold_change", legend_group="animal id",
          source=df_plot, marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size,
          color=bokeh.transform.factor_cmap("animal id", marker_colors, animal_id), line_color="black", line_width=marker_line_width)

############### ADD ERROR BARS ###################
source_error = bokeh.models.ColumnDataSource(data=df_plot)

w = bokeh.models.Whisker(source=source_error, base=bokeh.transform.dodge("base", -jitter_width, range=p.x_range), 
                            upper="upper", lower="lower", level="overlay", line_width=error_line_width)
w.upper_head.line_width=error_line_width
w.upper_head.size=error_size
w.lower_head.line_width=error_line_width
w.lower_head.size=error_size

p.add_layout(w)
############### ADD ERROR BARS ###################

p.y_range = bokeh.models.Range1d(-5, 80)
p.x_range = bokeh.models.Range1d(-2, 68)

ticks = df_lp_infant.loc[df_lp_infant["animal id"]=="RM-003", "base"].unique()+.5
labels = [variant + " DNA" for variant in list(df_lp_infant["variant"].unique())] + [variant + " RNA" for variant in list(df_lp_infant["variant"].unique())]

p.xaxis.ticker=ticks
p.xaxis.major_label_overrides = dict(zip(ticks, labels))

bokeh.io.show(p)

## Extended Data Figure 2b - Liver AAV barcode quantification (intra-cisterna magna delivery) 

In [5]:
############### BOKEH FIGURE SETTINGS ###################
tissue = "liver"
df_plot = df_lp_infant[df_lp_infant["tissue"]==tissue]
x_range = list(df_plot["cats"].unique())
animal_id = list(df_plot["animal id"].unique())
marker_colors = list(bokeh.palettes.Colorblind[5])
markers = ["circle_cross", "diamond_dot", "square_cross", "star", "cross"] 

figure_width = 700
figure_height = 350
bar_width = .75
bar_fill_color="gray"
jitter_width = 0.3
marker_size = 10
bar_line_width = 1

error_size=5
error_line_width=.25
marker_line_width=1

p = bokeh.plotting.figure(x_range=bokeh.models.FactorRange(*x_range), title="Liver AAV barcode quantification (infant rhesus macaque; intra-cisterna magna delivery)", 
                          height=figure_height, width=figure_width)

p.xgrid.visible=False
p.axis.minor_tick_line_width=0
p.xaxis.major_label_orientation=np.pi/4
p.axis.major_tick_in = 0
p.axis.major_label_text_color="#000000"
p.axis.major_label_text_font_size="12pt"
p.xaxis.group_text_font_size="12pt"
p.xaxis.group_text_color="#000000"
p.axis.major_label_standoff=5
p.add_layout(bokeh.models.Legend(), "right")
############### BOKEH FIGURE SETTINGS ###################

In [6]:
p.scatter(source=df_plot, x=bokeh.transform.jitter("cats", width=jitter_width, range=p.x_range), y="animal_mean_fold_change", legend_group="animal id",
          marker=bokeh.transform.factor_mark("animal id", markers, animal_id), size=marker_size, line_color="black", line_width=marker_line_width,
          color=bokeh.transform.factor_cmap("animal id", marker_colors, animal_id))

p.y_range = bokeh.models.Range1d(0, 3.5)
bokeh.io.show(p)